# Домашнее задание по математической статистике  
**Вариант 21**  

Проверка, снижает ли более длительное хранение содержание влаги в кирпичах.  
Уровень значимости $\alpha = 0.05$

In [11]:
import numpy as np

# исходные данные
month1 = np.array([7.0, 7.1, 7.1, 6.8, 7.2, 7.0, 7.2, 6.9, 6.7, 6.9])
month2 = np.array([5.7, 6.8, 5.1, 6.5, 6.1, 7.5, 7.1, 6.8, 6.9, 7.2,
                   6.2, 6.9, 7.0, 6.6, 6.0])

## 1. Выборочные средние  

$$ \bar x_1 = 6.99, \quad \bar x_2 = 6.56, \quad \bar x_{\text{общ}} = 6.732 $$

Здесь $\bar x_{\text{общ}}$ — общее среднее по 25 наблюдениям.

Формулы:
$$ \bar{x}_1 = \frac{1}{n_1} \sum_{i=1}^{n_1} x_i $$
$$ \bar{x}_2 = \frac{1}{n_2} \sum_{i=1}^{n_2} x_i $$

In [12]:
# проверяем средние в Python
mean1 = month1.mean(); mean2 = month2.mean();
overall_mean = (month1.sum()+month2.sum())/25
print(mean1, mean2, overall_mean)

6.99 6.5600000000000005 6.732


## 2. Пункт А. Критерий Фишера при нормальности

**Гипотезы:**  
$$
H_0: \mu_1 = \mu_2 \quad \text{(средняя влажность одинакова)} \\
H_1: \mu_1 > \mu_2 \quad \text{(после 2 месяцев влажность ниже)}
$$

**Статистика критерия Фишера** для двух нормальных выборок с разными дисперсиями:
$$
F = \frac{S_1^2 / n_1}{S_2^2 / n_2}
$$

Где:
- $S_1^2$, $S_2^2$ — выборочные дисперсии двух выборок,
- $n_1 = 10$, $n_2 = 15$ — объёмы выборок.


In [13]:
# Вычисления вручную
n1, n2 = len(month1), len(month2)
var1, var2 = np.var(month1, ddof=1), np.var(month2, ddof=1)

F_stat_manual = (var1 / n1) / (var2 / n2)
print(f"Дисперсия 1: {var1:.4f}, Дисперсия 2: {var2:.4f}")
print(f"F-статистика (ручной расчёт): {F_stat_manual:.4f}")


Дисперсия 1: 0.0277, Дисперсия 2: 0.4040
F-статистика (ручной расчёт): 0.1027


Для проверки гипотезы также используем встроенную функции `numpy` и `math`:

In [14]:
# Выборки
sample1 = np.array([7.0, 7.1, 7.1, 6.8, 7.2, 7.0, 7.2, 6.9, 6.7, 6.9])
sample2 = np.array([5.7, 6.8, 5.1, 6.5, 6.1, 7.5, 7.1, 6.8, 6.9, 7.2,
                    6.2, 6.9, 7.0, 6.6, 6.0])

# Параметры
mean1 = np.mean(sample1)
mean2 = np.mean(sample2)
var1 = np.var(sample1, ddof=1)
var2 = np.var(sample2, ddof=1)
n1 = len(sample1)
n2 = len(sample2)

# Расчет t-статистики (независимые выборки, неравные дисперсии — Welch’s t-test)
se = math.sqrt(var1 / n1 + var2 / n2)
t_stat = (mean1 - mean2) / se

# Степени свободы по формуле Уэлча
df = (var1 / n1 + var2 / n2) ** 2 / ((var1 ** 2) / (n1 ** 2 * (n1 - 1)) + (var2 ** 2) / (n2 ** 2 * (n2 - 1)))

# Уровень значимости
alpha = 0.05

# Критическое значение t для одностороннего теста
from math import erf, sqrt
def student_cdf(t, df):
    # Аппроксимация CDF Стьюдента через численные методы (очень приближенно, для строгости следует использовать таблицу)
    # Здесь можно использовать нормальное приближение
    return 0.5 * (1 + erf(t / sqrt(2)))

# Для df > 30 допустимо использовать нормальное приближение:
from scipy.stats import t as student
t_critical = student.ppf(1 - alpha, df)

# Сравниваем
print("t =", t_stat)
print("Критическое значение t (одностороннее):", t_critical)
if t_stat > t_critical:
    print("Отвергаем H0: вторая выборка имеет меньшее среднее")
else:
    print("Нет оснований отвергнуть H0")


t = 2.4951130686983345
Критическое значение t (одностороннее): 1.741107558723526
Отвергаем H0: вторая выборка имеет меньшее среднее


Если $p < 0.05$, то нулевая гипотеза отвергается — это означает, что после двух месяцев хранения влажность статистически значимо **ниже**.


## 3. Пункт Б. Критерий Краскала–Уоллиса

**Гипотезы:**  
$$
H_0: \text{Все выборки имеют одинаковое распределение} \\
H_1: \text{Есть статистически значимые различия в расположении выборок}
$$

Критерий Краскала–Уоллиса использует следующую статистику:
$$
H = \frac{12}{N(N+1)} \sum_{j=1}^{k} \frac{R_j^2}{n_j} - 3(N+1)
$$

Где:
- $R_j$ — сумма рангов в $j$-й группе,
- $n_j$ — размер $j$-й группы,
- $N = n_1 + n_2$ — общее количество наблюдений.



In [32]:
# Объединяем выборки
all_data = np.concatenate([month1, month2])

# Присваиваем ранги
ranks = np.argsort(np.argsort(all_data)) + 1

# Разделяем ранги обратно по группам
ranks_month1 = ranks[:len(month1)]
ranks_month2 = ranks[len(month1):]

# Суммы рангов
R1 = np.sum(ranks_month1)
R2 = np.sum(ranks_month2)

# Размеры
n1 = len(month1)
n2 = len(month2)
N = n1 + n2

# H-статистика
H = (12 / (N * (N + 1))) * ((R1 ** 2) / n1 + (R2 ** 2) / n2) - 3 * (N + 1)

# Расчёт p-value по распределению хи-квадрат с df = 1
# CDF хи-квадрат с 1 степенью свободы: F(x) = erf(sqrt(x/2))
p_value = 1 - math.erf(math.sqrt(H / 2))

print(f"H-статистика (вручную): {H:.4f}")
print(f"P-значение (вручную): {p_value:.4f}")

H-статистика (вручную): 3.7692
P-значение (вручную): 0.0522


Если $p < 0.05$, нулевая гипотеза отвергается: различие в содержании влаги **существенно** даже без предположения о нормальности распределений.


## 4. Итоговый вывод

На основе проведённых статистических проверок можно сделать следующие выводы:

- **Пункт А**: При предположении нормальности распределения и с помощью критерия Фишера
  (а также t-критерия Стьюдента) получено p-значение **< 0.05**, что позволяет отвергнуть
  нулевую гипотезу. Следовательно, **после 2 месяцев хранения содержание влаги статистически
  значимо меньше**, чем после 1 месяца.

- **Пункт Б:** При использовании непараметрического критерия Краскала–Уоллиса получено p-значение > 0.05, что **не позволяет отвергнуть нулевую гипотезу**. Следовательно, **при отказе от предположения о нормальности** статистически значимого различия между выборками **не обнаружено**.


**Вывод:** при использовании параметрических критериев (при предположении нормальности) мы наблюдаем статистически значимое снижение влажности при длительном хранении. Однако при непараметрическом подходе (без предположения нормальности) полученное различие **не является значимым на уровне 0.05**. Следовательно, вывод о снижении влажности **зависит от предположения о нормальности распределения**.

